In [ ]:

Google_Drive = True

Checkpoint_Google_Save = False

Google_Drive_Dir = 'webui'



Checkpoint_Default = 'https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors'

Checkpoint_DefaultName = 'Counterfeit-V3.0_fp16.safetensors'

Ngrok_Key = ''



ControlNet = True


DDetailer = True


SegmentAnything = True

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lora
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/embedding
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/wildcards

checkpoint_url = Checkpoint_Default
checkpoint_name = Checkpoint_DefaultName
checkpoint_down_path = '/content/sd-webui/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  checkpoint_down_path = '/content/sd-webui/models/Stable-diffusion/google'

checkpoint_file = checkpoint_down_path+'/'+checkpoint_name



%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2 libcairo2-dev
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

if DDetailer:
  !pip install -U openmim
  !mim install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
  !pip install mmdet==2.28.2

!git clone -b stable1 https://github.com/neuralninja22/sd-webui
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/sd-webui/models/ESRGAN -o 4x-UltraSharp.pth
if ControlNet:
  !git clone https://github.com/Mikubill/sd-webui-controlnet /content/sd-webui/extensions/sd-webui-controlnet
if DDetailer:
  !git clone https://github.com/ninjaneural/ddetailer /content/sd-webui/extensions/ddetailer
if SegmentAnything:
  !git clone https://github.com/ninjaneural/sd-webui-segment-anything /content/sd-webui/extensions/sd-webui-segment-anything
!git clone https://huggingface.co/embed/negative /content/sd-webui/embeddings/negative
!git clone https://github.com/camenduru/sd-webui-tunnels /content/sd-webui/extensions/sd-webui-tunnels
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/sd-webui/extensions/deforum-for-automatic1111-webui
!git clone -b stable1 https://github.com/neuralninja22/sd-webui-images-browser /content/sd-webui/extensions/sd-webui-images-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/sd-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/fkunn1326/openpose-editor /content/sd-webui/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib /content/sd-webui/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex /content/sd-webui/extensions/posex
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/sd-webui/extensions/multidiffusion-upscaler-for-automatic1111
!git clone https://github.com/adieyal/sd-dynamic-prompts /content/sd-webui/extensions/sd-dynamic-prompts
!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding /content/sd-webui/extensions/sd-dynamic-thresholding
!git clone https://github.com/neuralninja22/sd-webui-wd14-tagger /content/sd-webui/extensions/sd-webui-wd14-tagger
!git clone https://github.com/hnmr293/sd-webui-llul /content/sd-webui/extensions/sd-webui-llul
!git clone https://github.com/fishslot/video_loopback_for_webui /content/sd-webui/extensions/video_loopback_for_webui
!git clone https://github.com/Scholar01/sd-webui-mov2mov /content/sd-webui/extensions/sd-webui-mov2mov
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/sd-webui/extensions/a1111-sd-webui-tagcomplete
!git clone https://github.com/ninjaneural/sd-webui-video-util /content/sd-webui/extensions/sd-webui-video-util
!git clone https://github.com/camenduru/sd-civitai-browser /content/sd-webui/extensions/sd-civitai-browser
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon /content/sd-webui/extensions/a1111-sd-webui-locon
%cd /content/sd-webui
!git reset --hard

!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config.json -O /content/sd-webui/config.json

if Google_Drive:
  !mkdir /content/sd-webui/models/Lora
  !mkdir /content/sd-webui/models/hypernetworks
  !rm /content/sd-webui/embeddings/google
  !rm /content/sd-webui/models/Stable-diffusion/google
  !rm /content/sd-webui/models/Lora/google
  !rm /content/sd-webui/models/hypernetworks/google
  !rm /content/sd-webui/outputs
  !mv /content/sd-webui/outputs /content/sd-webui/outputs_backup
  !rm /content/sd-webui/extensions/sd-dynamic-prompts/wildcards
  !mv /content/sd-webui/extensions/sd-dynamic-prompts/wildcards /content/sd-webui/extensions/sd-dynamic-prompts/wildcards_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/embedding /content/sd-webui/embeddings/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint /content/sd-webui/models/Stable-diffusion/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lora /content/sd-webui/models/Lora/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork /content/sd-webui/models/hypernetworks/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/sd-webui/outputs
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/wildcards /content/sd-webui/extensions/sd-dynamic-prompts/wildcards
else:
  !rm /content/sd-webui/outputs


if ControlNet:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o diff_control_sd15_temporalnet_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd15v2.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_zoedepth_sd15v1.pth


if SegmentAnything:
  !mkdir /content/sd-webui/models/sam
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -d /content/sd-webui/models/sam -o sam_vit_h_4b8939.pth


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{checkpoint_url}" -d {checkpoint_down_path} -o {checkpoint_name}


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {checkpoint_down_path} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/sd-webui/models/VAE -o kl-f8-anime2.ckpt


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/amornlnw7/koreanDollLikeness_v15/resolve/main/koreanDollLikeness_v15.safetensors -d /content/sd-webui/models/Lora -o koreanDollLikeness_v15.safetensors


!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://moritz.pm/files/ulzzang-6500.pt -d /content/sd-webui/embeddings -o ulzzang-6500.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/sd-webui/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/sd-webui/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/sd-webui/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/sd-webui/embeddings/ -o bad_prompt_version2.pt

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/sd-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/sd-webui/launch.py

!mkdir /content/sd-webui/extensions/deforum-for-automatic1111-webui/models
!python launch.py  --theme dark --share --xformers --enable-insecure-extension-access --gradio-queue --cloudflared